In [1]:
%load_ext autoreload
%autoreload 2

## **Load Data**

In [7]:
import json
import requests
import pandas as pd
import yfinance as yf

In [38]:
config = {
    "datasets": {
        "SPY": {
            "type": "yahoo",
            "ticker": "SPY",
            "frequency": "D",
            "seasonality": 5,
            "start_date": "2020-01-01",
            "end_date": "2024-12-31"
        },
        "AQI_LA": {
            "type": "aqi",
            "lat": 34.0522,
            "lon": -118.2437,
            "frequency": "D",
            "seasonality": 7,
            "start_date": "2022-08-10",
            "end_date": "2024-12-31"
        }
    }
}

### **Yahoo Finance**

In [39]:
cfg = config['datasets']['SPY']
print(f"Downloading {cfg['ticker']} from Yahoo Finance...")

In [40]:
df_spy = yf.download(
    cfg["ticker"],
    start=cfg["start_date"],
    end=cfg["end_date"],
    interval="1d",
    auto_adjust=True,
    progress=False)
df_spy.head()

Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2020-01-02,298.578613,298.597013,296.427990,297.356261,59151200
2020-01-03,296.317810,297.448278,295.113829,295.168971,77709700
2020-01-06,297.448181,297.530894,294.433601,294.553085,55653900
2020-01-07,296.611847,297.356292,296.161483,296.878357,40496400
2020-01-08,298.192688,299.415040,296.556742,296.804881,68296000


In [41]:
if isinstance(df_spy.columns, pd.MultiIndex):
    if "Close" in df_spy.columns.levels[0]:
        df_spy = df_spy.xs("Close", axis=1, level=0)
    else:
        df_spy = df_spy.iloc[:, 0]
elif "Close" in df_spy.columns:
    df_spy = df_spy[['Close']]
    
df_spy.head()

Ticker,SPY
Date,
2020-01-02,298.578613
2020-01-03,296.317810
2020-01-06,297.448181
2020-01-07,296.611847
2020-01-08,298.192688


In [42]:
df_spy.columns = ['value']
df_spy = df_spy.ffill().bfill()
df_spy.index = pd.to_datetime(df_spy.index).tz_localize(None)
df_spy.head()

,value
Date,
2020-01-02,298.578613
2020-01-03,296.317810
2020-01-06,297.448181
2020-01-07,296.611847
2020-01-08,298.192688


In [43]:
df_spy = df_spy.asfreq("B")
df_spy["value"] = df_spy["value"].ffill()

In [44]:
df_spy.index.name = 'date'

In [45]:
df_spy.head()

,value
date,
2020-01-02,298.578613
2020-01-03,296.317810
2020-01-06,297.448181
2020-01-07,296.611847
2020-01-08,298.192688


### **AQI**

In [46]:
cfg = config['datasets']['AQI_LA']
print(f"Downloading AQI for lat={cfg['lat']}, lon={cfg['lon']}...")

In [47]:
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
    "latitude": cfg["lat"],
    "longitude": cfg["lon"],
    "start_date": cfg["start_date"],
    "end_date": cfg["end_date"],
    "hourly": "us_aqi",
    "timezone": "auto"
}

In [48]:
response = requests.get(url, params=params)
response.raise_for_status()

In [49]:
data = response.json()

In [50]:
df_aqi = pd.DataFrame({
    "date": pd.to_datetime(data["hourly"]["time"]),
    "value": data["hourly"]["us_aqi"]
})

In [51]:
df_aqi.head()

,date,value
0,2022-08-10 00:00:00,137
1,2022-08-10 01:00:00,137
2,2022-08-10 02:00:00,137
3,2022-08-10 03:00:00,137
4,2022-08-10 04:00:00,138


In [52]:
df_aqi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    21000 non-null  datetime64[ns]
 1   value   21000 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 328.3 KB


In [53]:
df_aqi.set_index("date", inplace=True)

In [54]:
df_aqi.head()

,value
date,
2022-08-10 00:00:00,137
2022-08-10 01:00:00,137
2022-08-10 02:00:00,137
2022-08-10 03:00:00,137
2022-08-10 04:00:00,138


In [55]:
df_aqi.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21000 entries, 2022-08-10 00:00:00 to 2024-12-31 23:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   value   21000 non-null  int64
dtypes: int64(1)
memory usage: 328.1 KB


In [56]:
df_aqi = df_aqi.resample("D").max()

In [57]:
df_aqi["value"] = df_aqi["value"].interpolate(method="linear")

In [58]:
df_aqi.head()

,value
date,
2022-08-10,277
2022-08-11,155
2022-08-12,291
2022-08-13,291
2022-08-14,290


In [59]:
df_aqi.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 875 entries, 2022-08-10 to 2024-12-31
Freq: D
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   value   875 non-null    int64
dtypes: int64(1)
memory usage: 13.7 KB
